In [1]:
import os
import openai

from dotenv import load_dotenv

load_dotenv("/home/jovyan/.env")

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

In [3]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [4]:
get_current_weather('boston')

'{"location": "boston", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [5]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [12]:
from openai import OpenAI, AzureOpenAI
import os
import json
 
api_base = "https://uat-dsmbrsvc.anthem.com/llmgateway/openai"
api_key = os.environ["OPENAI_API_KEY"]
 
os.environ["SSL_CERT_FILE"] = "./new_root.pem"
messages=[{"role": "user", "content": "What's the weather like in Boston?"}]
 
client = OpenAI(base_url=api_base,api_key=api_key)
 
model = "gpt-4o-0513"
completion = client.chat.completions.create(model=model,
                                            messages=messages,
                                            functions=functions)


In [13]:
completion

'{"id": "chatcmpl-9rlaIALb4BHOXK6gp8x2nw98NqKJS", "choices": [{"finish_reason": "function_call", "index": 0, "logprobs": null, "message": {"content": null, "role": "assistant", "function_call": {"arguments": "{\\"location\\":\\"Boston, MA\\"}", "name": "get_current_weather"}, "tool_calls": null}}], "created": 1722601326, "model": "gpt-4o-2024-05-13", "object": "chat.completion", "service_tier": null, "system_fingerprint": "fp_abc28019ad", "usage": {"completion_tokens": 17, "prompt_tokens": 79, "total_tokens": 96}, "llm-gateway-requestid": "7b909e4c5d044f448368991b1987cebe", "responseCode": 200, "responseDesc": "LLM Request Completed Successfully", "deployment_name": "gpt-4o-0513"}'

In [14]:
res = json.loads(completion)
# print(res)
print(res["choices"][0]['message']['content'])

None


In [16]:
args=res["choices"][0]["message"]['function_call']['arguments']

In [17]:
args

'{"location":"Boston, MA"}'

In [18]:
obs=get_current_weather(args)

In [19]:
obs

'{"location": "{\\"location\\":\\"Boston, MA\\"}", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [20]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-4o-0513",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

{"id": "chatcmpl-9rlq9HpC0dLsTCS37amsnp8eESk9e", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "Hello! How can I assist you today?", "role": "assistant", "function_call": null, "tool_calls": null}}], "created": 1722602309, "model": "gpt-4o-2024-05-13", "object": "chat.completion", "service_tier": null, "system_fingerprint": "fp_abc28019ad", "usage": {"completion_tokens": 10, "prompt_tokens": 74, "total_tokens": 84}, "llm-gateway-requestid": "cd6bdaacda10416bb68f63adacf73843", "responseCode": 200, "responseDesc": "LLM Request Completed Successfully", "deployment_name": "gpt-4o-0513"}


In [21]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = client.chat.completions.create(
    model="gpt-4o-0513",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

{"id": "chatcmpl-9rlqv3Q4nadRKBXBSlkrVSR60e5P3", "choices": [{"finish_reason": "function_call", "index": 0, "logprobs": null, "message": {"content": null, "role": "assistant", "function_call": {"arguments": "{\"location\":\"Boston, MA\"}", "name": "get_current_weather"}, "tool_calls": null}}], "created": 1722602357, "model": "gpt-4o-2024-05-13", "object": "chat.completion", "service_tier": null, "system_fingerprint": "fp_abc28019ad", "usage": {"completion_tokens": 17, "prompt_tokens": 78, "total_tokens": 95}, "llm-gateway-requestid": "e7971544b30848269627bc98a911b9b6", "responseCode": 200, "responseDesc": "LLM Request Completed Successfully", "deployment_name": "gpt-4o-0513"}


In [22]:
response = client.chat.completions.create(
    model="gpt-4o-0513",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{"id": "chatcmpl-9rlrNHBOhQGRiBte7XTldU6VKqF7O", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "Would you prefer the temperature in Celsius or Fahrenheit?", "role": "assistant", "function_call": null, "tool_calls": null}}], "created": 1722602385, "model": "gpt-4o-2024-05-13", "object": "chat.completion", "service_tier": null, "system_fingerprint": "fp_abc28019ad", "usage": {"completion_tokens": 10, "prompt_tokens": 79, "total_tokens": 89}, "llm-gateway-requestid": "80f19e0f45e24abfb8b93992729f1d0d", "responseCode": 200, "responseDesc": "LLM Request Completed Successfully", "deployment_name": "gpt-4o-0513"}


In [23]:
response = client.chat.completions.create(
    model="gpt-4o-0513",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}
)
print(response)

{"id": "chatcmpl-9rlsY10ivoUQBGlLAaDqF6ifPuFvk", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": null, "role": "assistant", "function_call": {"arguments": "{\"location\":\"Boston, MA\"}", "name": "get_current_weather"}, "tool_calls": null}}], "created": 1722602458, "model": "gpt-4o-2024-05-13", "object": "chat.completion", "service_tier": null, "system_fingerprint": "fp_abc28019ad", "usage": {"completion_tokens": 7, "prompt_tokens": 88, "total_tokens": 95}, "llm-gateway-requestid": "1bd005e797f64f6f81bed020c8da6df3", "responseCode": 200, "responseDesc": "LLM Request Completed Successfully", "deployment_name": "gpt-4o-0513"}


In [24]:
res = json.loads(response)

In [26]:
messages.append(res['choices'][0]['message'])

In [27]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 {'content': None,
  'role': 'assistant',
  'function_call': {'arguments': '{"location":"Boston, MA"}',
   'name': 'get_current_weather'},
  'tool_calls': None}]

In [29]:
args = json.loads(res["choices"][0]["message"]['function_call']['arguments'])
observation = get_current_weather(args)

In [30]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [31]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 {'content': None,
  'role': 'assistant',
  'function_call': {'arguments': '{"location":"Boston, MA"}',
   'name': 'get_current_weather'},
  'tool_calls': None},
 {'role': 'function',
  'name': 'get_current_weather',
  'content': '{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'}]

In [32]:
response = client.chat.completions.create(
    model="gpt-4o-0513",
    messages=messages
)
print(response)

{"id": "chatcmpl-9rlwwsm8CCKCNwU61bvS9WYZWUeqq", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "The current weather in Boston, MA is 72\u00b0F, with sunny and windy conditions.", "role": "assistant", "function_call": null, "tool_calls": null}}], "created": 1722602730, "model": "gpt-4o-2024-05-13", "object": "chat.completion", "service_tier": null, "system_fingerprint": "fp_abc28019ad", "usage": {"completion_tokens": 18, "prompt_tokens": 75, "total_tokens": 93}, "llm-gateway-requestid": "369e2124ab90448fb290da5ce93f122e", "responseCode": 200, "responseDesc": "LLM Request Completed Successfully", "deployment_name": "gpt-4o-0513"}
